# tests

## Motivaciones

En el desarrollo de aplicaciones web, probar manualmente cada cambio es una tarea complicada y propensa a errores. Modificar una parte del código puede tener efectos colaterales difíciles de predecir, por lo que necesitamos automatizar las pruebas. Existen varias estrategias para realizar pruebas automáticas:

- **Probar elementos concretos sin un plan definido.**
- **Desarrollo guiado por pruebas (Test Driven Development, TDD).**
- **Desarrollo guiado por comportamiento (Behaviour Driven Development, BDD).**
- **Desarrollo guiado por pruebas de aceptación (Acceptance Test Driven Development, ATDD).**

## Frameworks de Pruebas

Algunos de los frameworks más utilizados para realizar pruebas en JavaScript son:

- **Jasmine**
- **Mocha + Chai**
- **Jest**
- **Selenium**
- **Puppeteer**
- **Vitest**

### Instalación de Vitest

Para instalar Vitest, utilizamos el siguiente comando:

```bash
npm install -D vitest
```

Debemos crear archivos de prueba que tengan la extensión `.test.js`.

### Configuración en `package.json`

```json
{
  "scripts": {
    "test": "vitest",
    "coverage": "vitest run --coverage"
  }
}
```

### Maneras de Ejecutar las Pruebas

Podemos ejecutar las pruebas y obtener informes de cobertura de código con los siguientes comandos:

```bash
npm run test
npm run coverage
npx vitest --ui
```

También podemos agregar Vitest como una extensión en Visual Studio Code desde el siguiente enlace: [Vitest Explorer](https://marketplace.visualstudio.com/items?itemName=vitest.explorer).

### Sintaxis de las Pruebas

Vitest acepta pruebas similares a las de Mocha-Chai, Jasmine o Jest. Podemos usar `test` o `it` para definir pruebas, y `expect` o `assert` para realizar afirmaciones. Aquí algunos enlaces útiles:

- [Expect](https://vitest.dev/api/expect.html)
- [Assert](https://vitest.dev/api/assert.html)

## Desarrollo Guiado por Pruebas (TDD)

El TDD es parte de la metodología de Extreme Programming y se basa en realizar pruebas unitarias. Sigue tres leyes fundamentales:

1. No escribirás código de producción sin antes escribir una prueba que falle.
2. No escribirás más de una prueba unitaria suficiente para fallar (y no compilar es fallar).
3. No escribirás más código del necesario para hacer pasar la prueba.

### Ciclo Red - Green - Refactor

1. **Red:** Escribir una prueba que falle antes de implementar la funcionalidad.
2. **Green:** Implementar el código mínimo necesario para que la prueba pase.
3. **Refactor:** Mejorar el código una vez que la prueba pase.

Una vez completado el ciclo, comenzamos de nuevo con el siguiente requisito. En cada ciclo se pueden redefinir o agregar nuevos requisitos.

### Matchers

Algunos matchers importantes que podemos usar en nuestras pruebas son:

- **not:** `expect(something).not.toBe(true);`
- **toBe:** `expect(thing).toBe(realThing);`
- **toBeDefined:** `expect(result).toBeDefined();`
- **toBeInstanceOf:** `expect('foo').toBeInstanceOf(String);`
- **toBeNaN, toBeNull, toBeUndefined:** `expect(thing).toBeNaN();`
- **toContain:** `expect(array).toContain(anElement);`
- **toEqual:** `expect(bigObject).toEqual({"foo": ['bar', 'baz']});`
- **toMatch:** `expect("my string").toMatch(/string$/);`

### Pruebas de Promesas

Para probar funciones asíncronas, solo necesitamos agregar `async` / `await` en la función de callback de `it`.

#### Ejemplo de Pruebas de Promesas

```javascript
describe('Test Promesas', function() {
  describe('incrementar decrementar', function() {
    test('should return the number +1', async function() {
      expect(await incrementar(1)).toBe(2);
      expect(await incrementar(1000)).toBe(1001);
    });
    test('should return the number -1', async function() {
      expect(await decrementar(1)).toBe(0);
      expect(await decrementar(1000)).toBe(999);
    });
  });
});
```

## Mocking, Stubs, Fake, Dummy

A veces necesitamos datos ficticios para realizar pruebas. Esto puede ser útil para no afectar la base de datos en producción o para tener más control sobre los datos que se están probando.

### Mocking de Variables

```javascript
const initial_state = [[0,0,0],[0,0,0],[0,0,0]];
const example_state = [[0,1,0],[0,-1,0],[0,0,0]];
const full_state = [[-1,1,-1],[1,-1,1],[1,-1,1]];

describe('3 en raya', function() {
  describe('Funciones del juego', function() {
    test('TicTacToeGetInitialState Debe retornar un array 3x3 de 0s', function() {
      expect(alg.TicTacToeGetInitialState()).toEqual(initial_state);
    });
    test('TicTacToeGetNextState Debe retornar una copia con la jugada', function() {
      expect(alg.TicTacToeGetNextState(initial_state,2,1)).toEqual([[0,0,1],[0,0,0],[0,0,0]]);
      expect(alg.TicTacToeGetNextState(initial_state,3,-1)).toEqual([[0,0,0],[-1,0,0],[0,0,0]]);
      expect(alg.TicTacToeGetNextState(initial_state,2,1)).not.toBe(initial_state);
    });
    test('TicTacToeGetValidMoves Debe retornar la lista de posibles acciones', function() {
      expect(alg.TicTacToeGetValidMoves(initial_state)).toEqual([1,1,1,1,1,1,1,1,1]);
      expect(alg.TicTacToeGetValidMoves(example_state)).toEqual([1,0,1,1,0,1,1,1,1]);
      expect(alg.TicTacToeGetValidMoves(full_state)).toEqual([0,0,0,0,0,0,0,0,0]);
    });
  });
});
```

### Mocking de Funciones

A veces queremos saber si una función se ejecutó, aunque no siempre nos interesa su retorno. Vitest proporciona la utilidad `vi` para este propósito.

- **`vi.fn()`**: Crea una función simulada.
- **`vi.spyOn()`**: Añade un espía a una función de un objeto.

#### Ejemplo de Mocking de Funciones

```javascript
const myFunction = vi.fn();
myFunction();
expect(myFunction).toHaveBeenCalled();
```

### Mocking de la Red

Vitest recomienda utilizar [msw](https://mswjs.io/) para interceptar peticiones y crear respuestas falsas.

#### Ejemplo de Mocking de la Red

```javascript
import { http, HttpResponse } from 'msw';
import { setupServer } from 'msw/node';

const server = setupServer(
  http.get('https://test.com/rest/v1/fallo', (req, res, ctx) => {
    return HttpResponse.error();
  }),
);
```

### Mocking del Navegador

Vitest se ejecuta a través de la terminal, en Node.js. Para simular un navegador, se recomienda usar JSDOM. 

#### Instalación de JSDOM

```bash
npm install -D jsdom
```

Y agregamos el siguiente comentario al principio del archivo de pruebas:

```javascript
/**
 * @vitest-environment jsdom
 */
```

### Mocking del DOM

Para simular interacciones con el DOM, podemos utilizar [DOM Testing Library](https://testing-library.com/docs/dom-testing-library/example-intro).

### Pruebas End-to-End (E2E) y QA

Para pruebas de extremo a extremo, podemos utilizar herramientas como Cypress. Vitest proporciona una metodología más tradicional tipo “scripted testing”.

En resumen, Vitest es una herramienta poderosa y flexible para realizar pruebas automatizadas en aplicaciones JavaScript. Proporciona una amplia gama de funcionalidades para manejar pruebas unitarias, integraciones y simulaciones, lo que facilita la creación de pruebas robustas y efectivas para garantizar la calidad de nuestro código.